In [1]:
#potentially alternative to uploading
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/dlam_project
!pwd

Mounted at /content/drive
/content/drive/My Drive/dlam_project
/content/drive/My Drive/dlam_project


In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
#!ls -lha kaggle.json
#!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
#!cp kaggle.json ~/.kaggle/ # move the key to the folder
#!pwd # checking the present working directory
#!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!pip install -q kaggle

In [ ]:
#%cd /content
#!kaggle datasets download arevel/chess-games
#!unzip -qq /content/chess-games.zip
#%cd /content

In [1]:
#file_path = '/content/chess_games.csv'#
#file_path = '/content/drive/My Drive/dlam_project/chess_games.csv'
file_path = 'C:/Users/Nils/Downloads/archive/chess_games.csv'

In [3]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00


In [2]:
#IMPORTS
import numpy as np
import pandas as pd
import chess
import gc
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from neural_networks import CopiedPolicyChessNetwork, CopiedPolicyChessNetworkWithExtendedMoveRep
from datasets import Copied_Dataset, CopiedDatasetExtendedMoveRep



In [3]:
# SPECIFY DEFAULT SETTINGS
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_tensor_type(torch.DoubleTensor)

In [10]:
chess_data_raw = pd.read_csv(file_path, usecols=["AN","WhiteElo"], nrows=200000)
chess_data= chess_data_raw[chess_data_raw['WhiteElo'] > 2000]
del chess_data_raw
gc.collect()
chess_data = chess_data[['AN']]
chess_data = chess_data[~chess_data['AN'].str.contains('{')]
chess_data = chess_data[chess_data['AN'].str.len() > 20]
chess_data = chess_data.sample(frac=1)
print(chess_data.shape)


(27695, 1)


In [5]:
#board = chess.Board()
#dic = board.piece_map()
#for pos, piece in dic.items():
#    print(pos, piece.symbol())

#Gets the symbol P, N, B, R, Q or K for white pieces or the lower-case variants for the black pieces.

#board layout in storage
# 0 r n b q k b n r
# 1 p p p p p p p p
# 2 . . . . . . . .
# 3 . . . . . . . .
# 4 . . . . . . . .
# 5 . . . . . . . .
# 6 P P P P P P P P
# 7 R N B Q K B N R

# board layout in chess
# 8 r n b q k b n r
# 7 p p p p p p p p
# 6 . . . . . . . .
# 5 . . . . . . . .
# 4 . . . . . . . .
# 3 . . . . . . . .
# 2 P P P P P P P P
# 1 R N B Q K B N R
#   a b c d e f g h

# pos in piece_map is letter_tonum[letter] + 8 * number

In [11]:
N_train = 20000
N_test = 5000
N_epochs = 20
#data_train = Chess_Dataset(chess_data.iloc[:N_train],)
#data_loader_train = DataLoader(data_train, batch_size = 32, shuffle = False)
#
#data_test = Chess_Dataset(chess_data.iloc[N_train: N_train + N_test],)
#data_loader_test = DataLoader(data_test, batch_size = 32, shuffle = False)


#data_train = Copied_Dataset(chess_data, N_train)
#data_loader_train = DataLoader(data_train, batch_size = 32, shuffle = True)
#
#data_test = Copied_Dataset(chess_data, N_test)
#data_loader_test = DataLoader(data_test, batch_size = 32, shuffle = True)

data_train = CopiedDatasetExtendedMoveRep(chess_data, N_train)
data_loader_train = DataLoader(data_train, batch_size = 32, shuffle = True)

data_test = CopiedDatasetExtendedMoveRep(chess_data, N_test)
data_loader_test = DataLoader(data_test, batch_size = 32, shuffle = True)

# Training and Eval Loop

In [12]:
from torch.optim import lr_scheduler

In [15]:
lr = 3e-4
hidden_size = 10
hidden_layers = 4
model = CopiedPolicyChessNetworkWithExtendedMoveRep(hidden_ayers = hidden_layers, hidden_size = hidden_size).to(device)
metric_from = nn.CrossEntropyLoss()
metric_to = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)


In [16]:
model = model.to(device)
model.eval()
losses = []
for data, target in data_loader_test:
    with torch.no_grad():
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        loss_from = metric_from(output[:, 0:6, :, :], target[:, 0:6, :, :])
        loss_to = metric_to(output[:, 6:12, :, :], target[:, 6:12, :, :])
        loss = loss_from + loss_to
        losses.append(loss.cpu())
print(np.mean(losses))


for i in range(N_epochs):
  model.train()
  print(f"Epoch {i} starting ...")
  losses = []
  data_count = 0
  for data, target in data_loader_train:
      model.zero_grad()
      data = data.to(device)
      target = target.to(device)
      output = model(data)
      loss_from = metric_from(output[:, 0:6, :, :], target[:, 0:6, :, :])
      loss_to = metric_to(output[:, 6:12, :, :], target[:, 6:12, :, :])
      loss = loss_from + loss_to
      loss.backward()
      optimizer.step()
      if data_count % (data_loader_train.batch_size * 300) == 0:
        print(f"Datapoint {data_count} reached", end=", ")
      data_count += data_loader_train.batch_size
      losses.append(loss.detach().cpu().numpy())
  print("")
  print("Training Loss: ", np.mean(losses))
  model.eval()

  losses = []
  for data, target in data_loader_test:
    with torch.no_grad():
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        loss_from = metric_from(output[:, 0:6, :, :], target[:, 0:6, :, :])
        loss_to = metric_to(output[:, 6:12, :, :], target[:, 6:12, :, :])
        loss = loss_from + loss_to
        losses.append(loss.cpu())
  # Calculate validation loss on the development dataset
  validation_loss = np.mean(losses)
  scheduler.step(validation_loss)
  print("Evaluation Loss: ", np.mean(losses))
  torch.save(model.state_dict(), str(i) +'_model_weights_.pth')

0.061088122329125166
Epoch 0 starting ...
Datapoint 0 reached, Datapoint 9600 reached, Datapoint 19200 reached, 
Training Loss:  0.043669542797432744
Evaluation Loss:  0.03246987111962498
Epoch 1 starting ...
Datapoint 0 reached, Datapoint 9600 reached, Datapoint 19200 reached, 
Training Loss:  0.02773460089655698
Evaluation Loss:  0.02434080891519981
Epoch 2 starting ...
Datapoint 0 reached, Datapoint 9600 reached, Datapoint 19200 reached, 
Training Loss:  0.022295387660351072
Evaluation Loss:  0.020893766680513022
Epoch 3 starting ...
Datapoint 0 reached, Datapoint 9600 reached, Datapoint 19200 reached, 
Training Loss:  0.0197548891693958
Evaluation Loss:  0.018317350149146452
Epoch 4 starting ...
Datapoint 0 reached, Datapoint 9600 reached, Datapoint 19200 reached, 
Training Loss:  0.017838905095013784
Evaluation Loss:  0.017423741873115097
Epoch 5 starting ...
Datapoint 0 reached, Datapoint 9600 reached, Datapoint 19200 reached, 
Training Loss:  0.016773420778033777
Evaluation Loss

KeyboardInterrupt: 

In [12]:
#model = model.to(device)
#model.eval()
#losses = []
#for data, target in data_loader_test:
#    with torch.no_grad():
#        data = data.to(device)
#        target = target.to(device)
#        output = model(data)
#        loss_from = metric_from(output[:, 0, :], target[:, 0, :])
#        loss_to = metric_to(output[:, 1, :], target[:, 1, :])
#        loss = loss_from + loss_to
#        losses.append(loss.cpu())
#print(np.mean(losses))
#
#
#for i in range(N_epochs):
#  model.train()
#  print(f"Epoch {i} starting ...")
#  losses = []
#  data_count = 0
#  for data, target in data_loader_train:
#      model.zero_grad()
#      data = data.to(device)
#      target = target.to(device)
#      output = model(data)
#      loss_from = metric_from(output[:, 0, :], target[:, 0, :])
#      loss_to = metric_to(output[:, 1, :], target[:, 1, :])
#      loss = loss_from + loss_to
#      loss.backward()
#      optimizer.step()
#      if data_count % (data_loader_train.batch_size * 300) == 0:
#        print(f"Datapoint {data_count} reached", end=", ")
#      data_count += data_loader_train.batch_size
#      losses.append(loss.detach().cpu().numpy())
#  print("")
#  print("Training Loss: ", np.mean(losses))
#  model.eval()
#
#  losses = []
#  for data, target in data_loader_test:
#    with torch.no_grad():
#        data = data.to(device)
#        target = target.to(device)
#        output = model(data)
#        loss_from = metric_from(output[:, 0, :], target[:, 0, :])
#        loss_to = metric_to(output[:, 1, :], target[:, 1, :])
#        loss = loss_from + loss_to
#        losses.append(loss.cpu())
#  # Calculate validation loss on the development dataset
#  validation_loss = np.mean(losses)
#  scheduler.step(validation_loss)
#  print("Evaluation Loss: ", np.mean(losses))
#  torch.save(model.state_dict(), str(i) +'_model_weights_.pth')

In [ ]:
torch.save(model.state_dict(), 'model_weights_8000.pth')
#to load the weights again use:
#model = CopiedChessNetwork(hidden_layers=hidden_layers, hidden_size=hidden_size)
#model.load_state_dict(torch.load('model_weights_0dot15_loss.pth'))
